In [1]:
pip install openai

In [3]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [4]:
import os

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [8]:
import requests

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

In [7]:
get_weather_yiqiao(0, 0)

24.6

In [9]:
from openai import OpenAI
import json

client = OpenAI()

tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for provided coordinates in celsius.",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {"type": "number"},
            "longitude": {"type": "number"}
        },
        "required": ["latitude", "longitude"],
        "additionalProperties": False
    },
    "strict": True
}]

input_messages = [{"role": "user", "content": "What's the weather like in Paris today?"}]

response = client.responses.create(
    model="gpt-4.1",
    input=input_messages,
    tools=tools,
)

In [10]:
response

Response(id='resp_6860201f65288199b1cd5fb7af7ce1400985bab7b6826d3c', created_at=1751130143.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-2025-04-14', object='response', output=[ResponseFunctionToolCall(arguments='{"latitude":48.8566,"longitude":2.3522}', call_id='call_keEkC4eb9xLyF2Lph4ribdmJ', name='get_weather', type='function_call', id='fc_6860202015ac81999ac56a5514a958270985bab7b6826d3c', status='completed')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[FunctionTool(name='get_weather', parameters={'type': 'object', 'properties': {'latitude': {'type': 'number'}, 'longitude': {'type': 'number'}}, 'required': ['latitude', 'longitude'], 'additionalProperties': False}, strict=True, type='function', description='Get current temperature for provided coordinates in celsius.')], top_p=1.0, background=False, max_output_tokens=None, previous_response_id=None, prompt=None, reasoning=Reasoning(effort=None, generate_summary=None,

In [11]:
tool_call = response.output[0]
args = json.loads(tool_call.arguments)

In [12]:
args

{'latitude': 48.8566, 'longitude': 2.3522}

In [13]:
result = get_weather(args["latitude"], args["longitude"])

In [14]:
result

29.2

In [15]:
input_messages.append(tool_call)  # append model's function call message
input_messages.append({                               # append result message
    "type": "function_call_output",
    "call_id": tool_call.call_id,
    "output": str(result)
})

input_messages

[{'role': 'user', 'content': "What's the weather like in Paris today?"},
 ResponseFunctionToolCall(arguments='{"latitude":48.8566,"longitude":2.3522}', call_id='call_keEkC4eb9xLyF2Lph4ribdmJ', name='get_weather', type='function_call', id='fc_6860202015ac81999ac56a5514a958270985bab7b6826d3c', status='completed'),
 {'type': 'function_call_output',
  'call_id': 'call_keEkC4eb9xLyF2Lph4ribdmJ',
  'output': '29.2'}]

In [16]:
response_2 = client.responses.create(
    model="gpt-4.1",
    input=input_messages,
    tools=tools,
)
print(response_2.output_text)

The weather in Paris today is 29.2°C. It’s quite warm, so make sure to stay hydrated if you’re out and about!


```json
{
  "location": str,
  "temperature": float,
}
```